## ASG5 part2 - all files
          

Cluster the business sections (/blue/acg7849/share/BS) using Doc2Vec (50 clusters) in two ways:

Using a counter as the ‘tag’ (as in 5.1.5)

Using a counter as the ‘tag’, and the industry code as an additional tag (yield TaggedDocument(words=file_tokens, tags=[i, SIC]) where SIC is a string holding the tag (for example ‘1740’)

Extract the 4-digit SIC industry code from the annual report header (STANDARD INDUSTRIAL CLASSIFICATION).

Required: Evaluate whether adding the industry code as an additional tag improves the clustering. Use the standard deviation of profitability as a way to evaluate this. (Firms that are more similar, should have similar performance. Therefore, a better clustering would result in lower standard deviations for each cluster, relative to a worse clustering).

Do this for the filings for the year 2019 only. Calculate the standard deviation of performance for each cluster (use the year of CONFORMED END OF PERIOD, which are the first 4 digits of ‘date’ in summary.text).

For 50 clusters that means you will have 2 standard deviations for each cluster (one for each approach, with the extra SIC tag vs not adding the extra SIC tag). Use a t-test to test for a difference between the two sets of 50 standard deviations.

In [1]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import gensim
stopWords = set(stopwords.words('english') )
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import FreqDist
from nltk.collocations import *

/apps/jupyterhub/1.1.0/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# already made this - copy + paste to personal directory

#try copy and past targets
#think this is a pretty bold way avoiding messing up with the class

import shutil
for i in range(len(listoflink)):
    shutil.copy(listoflink[i], r'/blue/acg7849/hli1/item1')
    #shutil.copy(listoflink[i], r'/blue/acg7849/tbode/whateverfile')

# codes from 515 - start with new directory

In [2]:
# add some punctuation to string.punctuation
punc = string.punctuation + '“”'

# documents get tagged by an index (number), while filenames have different numbers
# keep track of this
fileIdToIndex = {} # given a fileId -> tag
indexToFileId=[] # given a tag -> fileId

class MyFiles(object):
    def __init__(self, dirname, tokens_only = False):
        self.dirname = dirname
        self.tokens_only = tokens_only
 
    def __iter__(self):
        #for i, fname in enumerate(files[0:200]):
         
        for i, fname in enumerate(os.listdir(self.dirname)[0:4604]):    # there are 4604 files in total
        # enumerate = return a list of tuples, iterate from start to end
        # os.listdir = return index of a directory, input = directory address
        # this part enumerates the first 200 units in the index under dirname
        
            with open( os.path.join(self.dirname, fname), encoding='utf-8') as f:
                content = f.read()
            # filter
                #content = [f for f in content if int(f["length"])> 1000 and (f["date"][0:4]) == '2019']
                #content = [f for f in content if len(f)> 1000]
            # grab id from filename
            myCounter = int ( re.findall(r'(\d*)\.txt', fname)[0] )
            # update 
            fileIdToIndex [ myCounter] = i
            indexToFileId.append( myCounter)
            #print('fname', fname, 'tag', myCounter)
            file_tokens = [x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation]
            
            if self.tokens_only == True:
                yield file_tokens
            else:
                yield TaggedDocument(words=file_tokens, tags=[i] )                    

In [3]:
# Hipergator
ffiles = MyFiles(r'/blue/acg7849/hli1/item1') # this one expected str, bytes or os.PathLike object
#ffiles = MyFiles(r'/blue/acg7849/share/BS/item1/') # a memory-friendly iterator
# dirname = '/blue/acg7849/share/BS/item1/'?

In [4]:
# create a model, build vocabulary
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=8)
model.build_vocab(ffiles) 

In [5]:
# train it
model.train(ffiles, total_examples=model.corpus_count, epochs=model.epochs)
print('FINISH')

FINISH


In [6]:
# Hipergator
def tokenizeFile(file_id):
    with open( r'/blue/acg7849/hli1/item1/'+str(file_id)+'.txt', encoding='utf-8') as f:
            content = f.read()
    return ([x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation] )

In [7]:
t = tokenizeFile(277350) # test file = 277350
model.infer_vector( t )

array([-1.248418  ,  1.4292227 , -1.6375788 ,  0.6757137 , -0.12051744,
       -2.3390086 , -0.5085806 , -0.26482967, -0.91265184, -0.22861241,
        1.7329948 ,  3.1974473 , -1.3469163 , -1.2657019 ,  0.10643362,
       -0.6741763 , -1.4705172 ,  1.43591   , -1.0513124 , -1.6309171 ,
       -0.43597904, -0.83159715, -0.26696193,  0.79424316, -2.7946398 ,
        2.032579  , -1.5442758 ,  2.0036602 , -2.544605  ,  0.51530355,
        0.01094823, -0.8459205 , -0.9021442 ,  0.1591185 , -1.2317147 ,
        0.9587873 , -3.4394996 , -3.436198  ,  1.3857045 , -1.2811754 ,
        0.2749075 , -1.5542    ,  0.3914699 ,  2.6439848 ,  0.56674767,
       -3.6509738 ,  0.04117782, -0.3678927 ,  4.5447173 , -0.03000459,
       -2.72903   ,  0.9888969 , -1.0949597 , -2.595117  , -0.97578937,
       -1.6974614 , -1.0381728 ,  1.1890405 , -1.3896854 ,  0.05372056,
        0.8162006 , -0.6583591 ,  2.5128868 , -1.1914207 ,  1.2334534 ,
       -3.4831507 ,  0.08891239, -0.74672735,  0.94904745, -0.98

In [8]:
t = tokenizeFile(277350)
inferred_vector = model.infer_vector( t )
# dv is short for docvecs
# most similar files to 277350 is the 4th one 265065?
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
#sims = model.dv.most_similar([inferred_vector], topn=4)
sims[0:5]

[(3711, 0.9831337928771973),
 (2789, 0.6813980937004089),
 (3302, 0.6781359314918518),
 (3153, 0.6496272087097168),
 (209, 0.6474929451942444)]

In [9]:
# letter with filename 1.txt is the first letter, so tag is 0
similar_doc = model.docvecs.most_similar(0)
similar_doc

<ipython-input-9-0bceb90a8426>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(0)


[(4321, 0.7674031853675842),
 (1806, 0.7462068200111389),
 (2554, 0.7375996708869934),
 (3588, 0.7360659241676331),
 (4225, 0.7337934970855713),
 (103, 0.7311334013938904),
 (324, 0.7248653769493103),
 (1924, 0.7222644686698914),
 (1656, 0.7208724617958069),
 (4586, 0.7205362915992737)]

In [10]:
print('number of documents', model.corpus_count)
print('model.docvecs', len(model.dv))

number of documents 4604
model.docvecs 4604


In [11]:
# Hipergator
# reread the files, and get the vector for each file
# feed vector into k-means algorithm to make clusters
wordLists = MyFiles(r'/blue/acg7849/hli1/item1', tokens_only = True) # a memory-friendly iterator
vectors = [ model.infer_vector( w ) for w in wordLists]
print(len(vectors))
vectors[0]

4604


array([ 0.14280093,  1.5915762 , -0.9604673 ,  0.5018748 ,  2.6790376 ,
        2.0797226 , -4.5312366 , -0.556001  , -0.6834529 ,  0.6691592 ,
        2.400295  ,  1.2024211 , -2.390524  , -1.872069  , -1.6160091 ,
       -0.8940099 ,  0.80020404, -0.6432669 , -2.1719573 , -2.7052314 ,
       -1.0866717 ,  2.2793481 , -1.9636626 ,  0.34861258,  0.49692783,
       -0.10679485, -5.4840097 ,  0.60280097, -0.10563369, -0.24827555,
        1.4929724 ,  0.833405  , -0.796235  , -0.13962702, -1.410608  ,
       -0.855484  ,  0.75842106, -0.5464739 ,  1.9920248 ,  1.2304072 ,
       -1.2357284 , -1.4421774 ,  2.1721776 ,  1.2087462 ,  1.5260332 ,
       -4.038274  , -0.7345342 , -0.83330774,  1.7719223 ,  0.2191632 ,
        0.10573795,  1.2381473 , -2.8514812 , -0.38032132, -1.608861  ,
       -1.8910835 ,  0.94259727,  2.0549424 ,  0.18705535,  1.3143021 ,
        0.39445272,  2.2172103 ,  0.09930221, -0.43579805,  0.31941307,
       -0.31177285,  0.9254448 , -0.06337421, -2.0259273 , -3.56

In [12]:
from nltk.cluster import KMeansClusterer
num_clusters = 50
kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)
assigned_clusters[0:10]

[25, 24, 41, 21, 2, 11, 3, 22, 41, 8]

In [13]:
import collections
print(collections.Counter(assigned_clusters))

Counter({34: 329, 30: 236, 10: 222, 11: 202, 21: 161, 40: 151, 48: 150, 45: 144, 29: 143, 41: 133, 42: 130, 31: 127, 46: 122, 32: 103, 4: 98, 16: 97, 3: 96, 25: 92, 8: 92, 49: 91, 33: 86, 7: 81, 26: 77, 0: 77, 35: 77, 37: 76, 12: 75, 27: 73, 24: 66, 18: 65, 43: 65, 9: 61, 13: 59, 5: 56, 17: 56, 2: 55, 39: 54, 15: 52, 1: 51, 19: 50, 36: 50, 44: 49, 38: 48, 47: 47, 14: 44, 22: 42, 23: 32, 20: 23, 6: 21, 28: 17})


# also needs: get industrial codes

In [14]:
SIC = filing[ filing.find('STANDARD INDUSTRIAL CLASSIFICATION:') + 35 : filing.find('IRS NUMBER') ]
SIC = ''.join(SIC.split())

NameError: name 'filing' is not defined